In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from sklearn import svm
import xgboost as xgb

from sklearn.metrics import accuracy_score

from scipy.stats import randint


In [2]:
data = pd.read_csv('./car.csv')
data = data.drop_duplicates()
data = data.drop(columns=['id', 'list_id', 'list_time', 'origin', 'seats', 'gearbox', 'color', 'mileage_v2'])
data

,manufacture_date,brand,model,type,fuel,price,condition
0,1980,Jeep,A2,SUV / Cross over,petrol,380000000.0,used
1,2021,Honda,City,Sedan,petrol,455000000.0,used
2,2016,Kia,Rio,NaN,petrol,295000000.0,used
3,2020,Toyota,Vios,Sedan,petrol,368000000.0,used
4,2001,Fiat,Siena,NaN,petrol,73000000.0,used
...,...,...,...,...,...,...,...
317611,2004,Ssangyong,Musso,SUV / Cross over,petrol,95000000.0,used
317613,2015,Kia,K3,Sedan,petrol,370000000.0,used
317617,2018,Hyundai,Grand i10,Hatchback,petrol,315000000.0,used
317618,2007,Hyundai,Veracruz,SUV / Cross over,oil,535000000.0,used


In [3]:
print(data.shape)
print(data.dtypes)

(81458, 7)
manufacture_date      int64
brand                object
model                object
type                 object
fuel                 object
price               float64
condition            object
dtype: object


In [4]:
data.describe()

,manufacture_date,price
count,81458.000000,8.142800e+04
mean,2015.880282,5.972179e+08
std,33.717270,6.841050e+08
min,-1.000000,1.310720e+06
25%,2014.000000,2.990000e+08
50%,2018.000000,4.690000e+08
75%,2021.000000,6.800000e+08
max,2023.000000,3.750000e+10


### **manufacture_date**

In [5]:
print(f'Number of NaN: {data["manufacture_date"].isna().sum()}')
print(f'Unique values: {data["manufacture_date"].unique()}')

Number of NaN: 0
Unique values: [1980 2021 2016 2020 2001 2022 2018 2009 2015 2008 2023 2007 2004 2019
 2014 2010 2012 1995 2003 2011 2017 2006 1999 2013 1994 1992 2000 2002
 1996 2005 1998 1997 1986   -1 1990 1985 1989 1984 1993 1988 1991 1987
 1983 1982]


Check rows with `manufacture_data = -1`

In [6]:
selected_rows = data[data['manufacture_date'] == -1]
selected_rows

,manufacture_date,brand,model,type,fuel,price,condition
1727,-1,NaN,NaN,NaN,NaN,190000000.0,used
4455,-1,NaN,NaN,NaN,NaN,190000000.0,used
5525,-1,NaN,NaN,NaN,NaN,190000000.0,used
6108,-1,NaN,NaN,NaN,NaN,190000000.0,used
6744,-1,NaN,NaN,NaN,NaN,190000000.0,used
44958,-1,NaN,NaN,NaN,NaN,890000000.0,used
87030,-1,NaN,NaN,NaN,NaN,890000000.0,used
102984,-1,NaN,NaN,NaN,NaN,890000000.0,used
116660,-1,NaN,NaN,NaN,NaN,890000000.0,used
147688,-1,NaN,NaN,NaN,NaN,890000000.0,used


In [7]:
data = data[data['manufacture_date'] != -1]
data

,manufacture_date,brand,model,type,fuel,price,condition
0,1980,Jeep,A2,SUV / Cross over,petrol,380000000.0,used
1,2021,Honda,City,Sedan,petrol,455000000.0,used
2,2016,Kia,Rio,NaN,petrol,295000000.0,used
3,2020,Toyota,Vios,Sedan,petrol,368000000.0,used
4,2001,Fiat,Siena,NaN,petrol,73000000.0,used
...,...,...,...,...,...,...,...
317611,2004,Ssangyong,Musso,SUV / Cross over,petrol,95000000.0,used
317613,2015,Kia,K3,Sedan,petrol,370000000.0,used
317617,2018,Hyundai,Grand i10,Hatchback,petrol,315000000.0,used
317618,2007,Hyundai,Veracruz,SUV / Cross over,oil,535000000.0,used


### **brand**

In [8]:
print(f'Number of NaN: {data["brand"].isna().sum()}')
print(f'Number of unique values: {data["brand"].nunique()}')
print(f'Unique values: {sorted(data["brand"].unique())}')

Number of NaN: 0
Number of unique values: 67
Unique values: ['Acura', 'Asia', 'Aston Martin', 'Audi', 'BMW', 'BYD', 'Baic', 'Bentley', 'Buick', 'Cadillac', 'Changan', 'Chery', 'Chevrolet', 'Chrysler', 'Citroen', 'Daewoo', 'Daihatsu', 'Dodge', 'Ferrari', 'Fiat', 'Ford', 'Gaz', 'Haima', 'Haval', 'Hãng khác', 'Honda', 'HongQi', 'Hyundai', 'Infiniti', 'Isuzu', 'Jaguar', 'Jeep', 'Kia', 'Lada', 'Lamborghini', 'LandRover', 'Lexus', 'Lifan', 'Lincoln', 'Luxgen', 'MG', 'Maserati', 'Maybach', 'Mazda', 'Mekong', 'Mercedes Benz', 'Mini', 'Mitsubishi', 'Nissan', 'Peugeot', 'Porsche', 'Proton', 'RAM', 'Renault', 'Rolls Royce', 'Rover', 'SYM', 'Samsung', 'Smart', 'Ssangyong', 'Subaru', 'Suzuki', 'Toyota', 'Vinfast', 'Volkswagen', 'Volvo', 'Zotye']


### **model**

In [9]:
print(f'Number of NaN: {data["model"].isna().sum()}')
print(f'Number of unique values: {data["model"].nunique()}')
print(f'Unique values: {sorted(data["model"].unique())}')

Number of NaN: 0
Number of unique values: 521
Unique values: ['1 Series', '107', '1083', '1500', '190', '2', '2 Series', '200', '2008', '205', '206', '208', '2108', '3', '3 Series', '3000GT', '3008', '300C', '305', '308', '323', '4 Runner', '4 Series', '404', '405', '408', '5', '5 Series', '500', '5008', '505', '508', '520', '6', '6 Series', '620', '626', '7', '7 MPV', '7 SUV', '7 Series', '718', '86', '911', '929', '940', 'A Class', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'AMG', 'APV', 'Accent', 'Accord', 'Acononline', 'Aerio', 'Albea', 'Almera', 'Alphard', 'Altima', 'Alto', 'Amigo', 'Aranos', 'Aristo', 'Aspire', 'Astro', 'Atenza', 'Atos', 'Attrage', 'Aurion', 'Avalon', 'Avanlanche', 'Avante', 'Avanza', 'Aveo', 'Aygo', 'Azera', 'B Class', 'BR-V', 'BT 50', 'Balenno', 'Beetle', 'Beijing X7', 'Bluebird', 'Boxster', 'Brio', 'C Class', 'CD5', 'CJ', 'CL Class', 'CLA Class', 'CLK Class', 'CLS Class', 'CR V', 'CR X', 'CTS', 'CX 3', 'CX 5', 'CX 8', 'CX 9', 'CX-30', 'Cadenza', 'Caldina'

### **type**

In [10]:
print(f'Number of NaN: {data["type"].isna().sum()}')
print(f'Number of unique values: {data["type"].nunique()}')
print(f'Unique values: {data["type"].unique()}')

Number of NaN: 11126
Number of unique values: 9
Unique values: ['SUV / Cross over' 'Sedan' nan 'Minivan (MPV)' 'Van' 'Hatchback'
 'Pick-up (bán tải)' 'Coupe (2 cửa)' 'Kiểu dáng khác' 'Mui trần']


In [11]:
# Fill Nan with the most frequent value
data['type'] = data['type'].fillna(data['type'].mode()[0])

C:\Users\DTBKhanh\AppData\Local\Temp\ipykernel_12248\3301231890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['type'] = data['type'].fillna(data['type'].mode()[0])


### **fuel**

In [12]:
print(f'Number of NaN: {data["fuel"].isna().sum()}')
print(f'Number of unique values: {data["fuel"].nunique()}')
print(f'Unique values: {data["fuel"].unique()}')

Number of NaN: 14
Number of unique values: 4
Unique values: ['petrol' 'oil' 'electric' 'hybrid' nan]


In [13]:
# Fill Nan with the most frequent value
data['fuel'] = data['fuel'].fillna(data['fuel'].mode()[0])

C:\Users\DTBKhanh\AppData\Local\Temp\ipykernel_12248\1156484555.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['fuel'] = data['fuel'].fillna(data['fuel'].mode()[0])


### **condition**

In [14]:
print(f'Number of NaN: {data["condition"].isna().sum()}')
print(f'Number of unique values: {data["condition"].nunique()}')
print(f'Unique values: {data["condition"].unique()}')

Number of NaN: 0
Number of unique values: 2
Unique values: ['used' 'new']


### **price**

In [15]:
print(f'Number of NaN: {data["price"].isna().sum()}')
print(f'Number of unique values: {data["price"].nunique()}')
print(f'Unique values: {data["price"].unique()}')

Number of NaN: 30
Number of unique values: 2095
Unique values: [3.80000e+08 4.55000e+08 2.95000e+08 ... 2.58000e+07 1.45555e+08
 1.23409e+09]


In [16]:
# Fill Nan with the mean value
data['price'] = data['price'].fillna(data['price'].mean())

C:\Users\DTBKhanh\AppData\Local\Temp\ipykernel_12248\2732459873.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['price'].fillna(data['price'].mean())


In [17]:
# Rename type column to type_car
data = data.rename(columns={'type': 'type_car'})

### **Save data**

In [18]:
data.to_csv('./car_final.csv')